In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score, precision_score, recall_score,f1_score, roc_auc_score


In [2]:
df=pd.read_csv("Fraud_data.csv")

# Preprocesssing of data


In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:
df.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.0,C2080388513,0.00,0.00,1,0
6362619,743,CASH_OUT,850002.52,C1280323807,850002.52,0.0,C873221189,6510099.11,7360101.63,1,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [6]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [7]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [8]:
df.shape

(6362620, 11)

# Down Sample the data

In [9]:
fraud_df = df[df['isFraud'] == 1]  # fraud cases
non_fraud_df = df[df['isFraud'] == 0]  # non-fraud cases

# Randomly sample % of non-fraud cases
non_fraud_sample = non_fraud_df.sample(frac=0.028, random_state=42)

# Combine fraud and sampled non-fraud data
balanced_df = pd.concat([fraud_df, non_fraud_sample])

In [10]:
balanced_df.shape

(186136, 11)

In [11]:
balanced_df["isFraud"].value_counts()

isFraud
0    177923
1      8213
Name: count, dtype: int64

In [12]:
balanced_df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

# Drop Unneceessary column

In [13]:
balanced_df.drop(['nameOrig', 'nameDest','isFlaggedFraud'], axis=1, inplace=True)

In [14]:
balanced_df.columns

Index(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'isFraud'],
      dtype='object')

In [18]:
balanced_df['type'].head(10)

2       3
3       0
251     3
252     0
680     3
681     0
724     0
969     3
970     0
1115    3
Name: type, dtype: int64

# Label Encoding to categorical column

In [16]:
balanced_df['type'].replace({'CASH_OUT':0, 'PAYMENT':1, 'CASH_IN':2, 'TRANSFER':3, 'DEBIT':4}, inplace=True)

In [17]:
balanced_df['type'].value_counts()

type
0    66555
1    60459
2    38879
3    19052
4     1191
Name: count, dtype: int64

In [19]:
balanced_df.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
2,1,3,181.0,181.0,0.0,0.0,0.0,1
3,1,0,181.0,181.0,0.0,21182.0,0.0,1
251,1,3,2806.0,2806.0,0.0,0.0,0.0,1
252,1,0,2806.0,2806.0,0.0,26202.0,0.0,1
680,1,3,20128.0,20128.0,0.0,0.0,0.0,1


# Splitting and Training , Testing

In [21]:
# X Data
x = balanced_df.drop(['isFraud'], axis=1)
print('X shape is : ' , x.shape)
print()

# y Data
y = balanced_df['isFraud']
print('Y shape is : ' , y.shape)

X shape is :  (186136, 7)

Y shape is :  (186136,)


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)

In [23]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((130295, 7), (55841, 7), (130295,), (55841,))

# Applying ML model 

In [24]:
Model_LR = LogisticRegression(class_weight='balanced')
Model_LR.fit(X_train, y_train)


LogisticRegression(class_weight='balanced')

In [25]:
y_pred_LR = Model_LR.predict(X_test)


In [26]:
Train_Accuracy = Model_LR.score(X_train, y_train)
Test_Accuracy = Model_LR.score(X_test, y_test)
print(f'Training accuracy: {Train_Accuracy*100:.2f} %')
print(f'Testing accuracy: {Test_Accuracy*100:.2f} %')

Training accuracy: 90.88 %
Testing accuracy: 91.28 %


In [27]:
print(classification_report(y_test,y_pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.91      0.95     53377
           1       0.32      0.90      0.48      2464

    accuracy                           0.91     55841
   macro avg       0.66      0.91      0.71     55841
weighted avg       0.97      0.91      0.93     55841



In [28]:
print(confusion_matrix(y_test,y_pred_LR))

[[48763  4614]
 [  255  2209]]


In [29]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
rf = RandomForestClassifier(max_samples = 0.8, oob_score = True,
                       random_state=7, class_weight='balanced')

In [35]:
rf.fit(X_train, y_train)



RandomForestClassifier(class_weight='balanced', max_samples=0.8, oob_score=True,
                       random_state=7)

In [36]:
y_pred_rf = rf.predict(X_test)

In [40]:

Train_Accuracy = rf.score(X_train, y_train)
Test_Accuracy = rf.score(X_test, y_test)
print(f'Training accuracy: {Train_Accuracy*100:.2f} %')
print(f'Testing accuracy: {Test_Accuracy*100:.2f} %')

Training accuracy: 100.00 %
Testing accuracy: 99.64 %


In [41]:
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     53377
           1       0.98      0.93      0.96      2464

    accuracy                           1.00     55841
   macro avg       0.99      0.97      0.98     55841
weighted avg       1.00      1.00      1.00     55841



In [42]:
print(confusion_matrix(y_test,y_pred_rf))

[[53341    36]
 [  167  2297]]


In [45]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(class_weight='balanced')
dt.fit(X_train, y_train)


DecisionTreeClassifier(class_weight='balanced')

In [46]:
y_pred_dt = dt.predict(X_test)

In [47]:

Train_Accuracy = dt.score(X_train, y_train)
Test_Accuracy = dt.score(X_test, y_test)
print(f'Training accuracy: {Train_Accuracy*100:.2f} %')
print(f'Testing accuracy: {Test_Accuracy*100:.2f} %')

Training accuracy: 100.00 %
Testing accuracy: 99.64 %


In [50]:
print(classification_report(y_test,y_pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     53377
           1       0.97      0.95      0.96      2464

    accuracy                           1.00     55841
   macro avg       0.98      0.98      0.98     55841
weighted avg       1.00      1.00      1.00     55841



In [51]:
print(confusion_matrix(y_test,y_pred_dt))

[[53294    83]
 [  117  2347]]


In [53]:
Recall_LR = recall_score(y_test, y_pred_LR)

In [54]:
Recall_rf = recall_score(y_test, y_pred_rf)

In [55]:
Recall_dt = recall_score(y_test, y_pred_dt)

In [56]:
evaluation = pd.DataFrame({'Classification Model': ['Logistic Regression','Decision Tree', 'Random Forest'],
                           'Accuracy Rate': [(Recall_LR*100).round(2), (Recall_dt*100).round(2), (Recall_rf*100).round(2)]})

evaluation

,Classification Model,Accuracy Rate
0,Logistic Regression,89.65
1,Decision Tree,95.25
2,Random Forest,93.22
